In [1]:
import json
import networkx as nx
from qa.generator import QAGenerator
from qa.templates import TEMPLATES

In [2]:
graph = nx.read_graphml("../data/municipalities_peaks_castles.graphml")

In [3]:
generator = QAGenerator()
qas = generator.generate_questions(graph, TEMPLATES, num_questions=1, add_distractors=10)
for qa in qas:
	print(qa)
	for fact in qa.context_facts:
		print(fact)

Generated 1 questions after 1 attempts.
QA(question='V kateri občini leži vrh Vajnež?', answer='Občina Jesenice', num_facts=11)
<Občina Jesenice> <ima_vrh> <Ptičji vrh>.
<Občina Jesenice> <ima_vrh> <Golica>.
<Občina Jesenice> <ima_vrh> <Ledinski vrh>.
<Vajnež> <se_nahaja_v> <Občina Jesenice>.
<Vajnež> <ima_višino> <2104.0>.
<Občina Jesenice> <ima_vrh> <Veliki vrh>.
<Občina Jesenice> <ima_vrh> <Potoški Stol>.
<Občina Jesenice> <ima_vrh> <Vajnež>.
<Občina Jesenice> <ima_populacijo> <21519>.
<Občina Jesenice> <ima_vrh> <Jesenice>.
<Občina Jesenice> <ima_površino> <75.8>.


In [4]:
generator = QAGenerator()
qas = generator.generate_questions(graph, TEMPLATES, num_questions=10000)

with open("../data/finetune_data.json", "w") as f:
	json.dump([{"prompt": qa.question, "response": qa.answer} for qa in qas], f, indent=2)

Generated 10000 questions after 10927 attempts.
